In [ ]:
#Method A - Try Classification Decision Tree with no max depth
#1 Load in Kaggle training data
#2 Train/Test Split
#3 Vectorize
#4 Instantiate 

#1 - Load in Kaggle training dataset
import pandas as pd
from matplotlib import pyplot as plt
import json
import numpy as np
import scipy as sp
from pprint import pprint
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import dummy
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
from sklearn import tree
from sklearn.tree import export_graphviz
from graphviz import Source
from IPython.display import SVG
from IPython.display import Image  
import pydotplus
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

with open('/Users/alirossi/Desktop/recipe-ingredients-dataset/train.json') as f:
    dict_train = json.load(f)

recipe = pd.DataFrame(dict_train)
recipe.head()

In [ ]:
#Create X (which contains dummy variables for each ingredient) and y

X = pd.get_dummies(recipe['ingredients'].apply(pd.Series).stack()).sum(level=0)
y = recipe['cuisine']

In [ ]:
#2 Train/Test Split - split into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=31)

In [ ]:
#Calculate null accuracy - use Dummy Classifier, predict all recipes are Italian 
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy='most_frequent', random_state=31)
dummy.fit(X_train, y_train)
dummy.score(X_test, y_test)

In [ ]:
#3 Classification Decision Tree with no max depth
from sklearn.tree import DecisionTreeClassifier

treeclf = DecisionTreeClassifier(random_state=31)
treeclf.fit(X_train, y_train)

In [ ]:
pd.DataFrame({'feature':X_train.columns, 'importance':treeclf.feature_importances_})\
    .sort_values('importance', ascending=False)

In [ ]:
#Source(tree.export_graphviz(treeclf, out_file='tree_recipe.png', feature_names=X_train.columns))
#export_graphviz(treeclf, out_file='tree_recipe.svg', feature_names=X_train.columns, max_depth=2)
graph = Source(tree.export_graphviz(treeclf, out_file=None, feature_names=X_train.columns, max_depth=5))
SVG(graph.pipe(format='svg'))

In [ ]:
#Need to test w test set
y_pred = treeclf.predict(X_test)

y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix   
print(classification_report(y_test, y_pred))
#precision = % true positive out of all predicted positives - 55% of Brazilian predictions were true
#recall = % true positive out of all actual positives - 43% of Brazilian recipes were predicted to be Brazilian

In [ ]:
print(confusion_matrix(y_test, y_pred, labels=['brazilian','british','cajun_creole','chinese','filipino','french'\
                                              ,'greek','indian','irish','italian','jamaican','japanese','korean',\
                                              'mexican','moroccan','russian','southern_us','spanish','thai',\
                                              'vietnamese'])) 
#header row = class predicted by model
#first column = actual classes

In [ ]:
#B - Classification Decision Tree with optimized max depth

In [ ]:
treeclf = DecisionTreeClassifier(random_state=31, max_depth=20)
treeclf.fit(X_train, y_train)

In [ ]:
pd.DataFrame({'feature':X_train.columns, 'importance':treeclf.feature_importances_})\
    .sort_values('importance', ascending=False)

In [ ]:
#Need to test w test set
y_pred = treeclf.predict(X_test)

y_pred

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix   
print(classification_report(y_test, y_pred))
#precision = % true positive out of all predicted positives 
#recall = % true positive out of all actual positives
#Not much better playing with max depth - now the less-represented recipes are not being predicted at all since their
#ingredients are not being used in the model

In [ ]:
#C - Random Forest

In [ ]:
# max_features=5 is best and n_estimators=150 is sufficiently large.
rfclf = RandomForestClassifier(n_estimators=150, max_features=None, oob_score=True, random_state=31)
rfclf.fit(X_train, y_train)

In [ ]:
# Compute feature importances.
pd.DataFrame({'feature':feature_cols, 'importance':rfreg.feature_importances_}).sort_values(by='importance')

In [ ]:
#Things we can try:
# - Classification Decision Tree x
# - Random Forest
# - As opposed to looking at discrete ingredients, just look at words/n-grams in ingredient list
# - Remove spaces to make ingredients all one word
# - Predict with % probabilities
# - Josh M. suggestion - use TFIDF (or LDA)